In [1]:
import os
import re  # <--- [FIX 1] Thêm thư viện re để xử lý regex
from docx import Document
from pypdf import PdfReader  # <--- [FIX 2] Dùng pypdf thay vì pdfminer để khớp với logic code của bạn

def read_input_file(input_path):
    """Hàm đọc file thông minh: Hỗ trợ cả PDF và DOCX"""
    print(f"📂 Đang đọc file nguồn: {input_path}...")
    
    # Kiểm tra file có tồn tại không
    if not os.path.exists(input_path):
        print(f"❌ Lỗi: File không tồn tại tại {input_path}")
        return None

    file_ext = os.path.splitext(input_path)[1].lower()
    raw_text = "" # Khởi tạo biến để tránh lỗi UnboundLocalError

    try:
        full_text = []
        
        # --- XỬ LÝ FILE PDF ---
        if file_ext == '.pdf':
            # Logic này dùng thư viện pypdf
            reader = PdfReader(input_path)
            print(f"   📄 File PDF có {len(reader.pages)} trang.")
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    full_text.append(text)
            raw_text = "\n".join(full_text)

        # --- XỬ LÝ FILE DOCX ---
        elif file_ext == '.docx':
            doc = Document(input_path)
            for para in doc.paragraphs:
                # Xử lý các ký tự backslash thừa nếu có
                clean_line = re.sub(r'\\', '', para.text).strip()
                if clean_line:
                    full_text.append(clean_line)
            raw_text = "\n".join(full_text)
        
        else:
            raise ValueError("Định dạng file không được hỗ trợ (Chỉ chấp nhận .pdf hoặc .docx)")

        print("✅ Đã đọc xong văn bản nguồn.")
        return raw_text  # <--- [FIX 3] Phải return giá trị để sử dụng bên ngoài
        
    except Exception as e:
        print(f"❌ Lỗi đọc file: {e}")
        return None

# process ND100

In [3]:
def split_into_articles(raw_text):
        articles = []
        pattern = re.compile(r'(^|\n)(Điều\s+\d+[\.:].*?)')
        parts = pattern.split(raw_text)
        current_article = ""
        for part in parts:
            if re.match(r'Điều\s+\d+', part.strip()):
                if current_article:
                    articles.append(current_article)
                current_article = part
            else:
                current_article += part
        if current_article:
            articles.append(current_article)
        return articles

In [6]:
vehicle_patterns = {
    # Primary vehicle types
    "Xe ô tô": {
        "keywords": ["ô tô", "xe hơi", "car", "xe con"],
        "patterns": [r"xe\s+ô\s+tô", r"ô\s+tô", r"xe\s+hơi"],
        "priority": 10
    },
    "Xe mô tô, xe máy": {
        "keywords": ["mô tô", "xe máy", "xe gắn máy", "motorbike", "motorcycle"],
        "patterns": [r"xe\s+mô\s+tô", r"mô\s+tô", r"xe\s+máy", r"xe\s+gắn\s+máy"],
        "priority": 10
    },
    "Xe thô sơ": {
        "keywords": ["xe thô sơ", "xe bò", "sức người", "sức súc vật", "súc vật"],
        "patterns": [r"xe\s+thô\s+sơ", r"xe\s+bò", r"sức\s+người", r"sức\s+súc\s+vật", r"súc\s+vật"],
        "priority": 10
    },
    
    # Commercial vehicles
    "Xe khách, xe buýt": {
        "keywords": ["xe khách", "xe buýt", "xe bus", "bus", "buýt"],
        "patterns": [r"xe\s+khách", r"xe\s+buýt", r"xe\s+bus"],
        "priority": 9
    },
    "Xe tải, container": {
        "keywords": ["xe tải", "container", "xe container", "tải"],
        "patterns": [r"xe\s+tải", r"container"],
        "priority": 9
    },
    "Rơ moóc, sơ mi rơ moóc": {
        "keywords": ["rơ moóc", "moóc", "sơ mi rơ moóc", "trailer"],
        "patterns": [r"rơ\s+moóc", r"sơ\s+mi\s+rơ\s+moóc"],
        "priority": 9
    },
    
    # Special vehicles
    "Xe chuyên dụng": {
        "keywords": ["cứu thương", "cứu hỏa", "xe cứu", "xe cẩu", "xe ben", "xe bồn", "chuyên dụng"],
        "patterns": [r"cứu\s+thương", r"cứu\s+hỏa", r"xe\s+cứu", r"xe\s+cẩu", r"xe\s+ben", r"xe\s+bồn", r"chuyên\s+dụng"],
        "priority": 9
    },
    "Taxi, xe du lịch": {
        "keywords": ["taxi", "xe taxi", "xe du lịch", "du lịch"],
        "patterns": [r"taxi", r"xe\s+taxi", r"xe\s+du\s+lịch"],
        "priority": 8
    },
    
    # Other vehicles
    "Xe đạp": {
        "keywords": ["xe đạp", "bicycle", "bike", "đạp xe"],
        "patterns": [r"xe\s+đạp", r"đạp\s+xe"],
        "priority": 8
    },
    "Người đi bộ": {
        "keywords": ["người đi bộ", "pedestrian", "đi bộ"],
        "patterns": [r"người\s+đi\s+bộ", r"đi\s+bộ"],
        "priority": 8
    },
    "Xe lăn": {
        "keywords": ["xe lăn", "wheelchair", "người khuyết tật"],
        "patterns": [r"xe\s+lăn", r"người\s+khuyết\s+tật"],
        "priority": 8
    },
    
    # Railway and water transport
    "Tàu hỏa, đường sắt": {
        "keywords": ["tàu hỏa", "đường sắt", "railway", "train", "tàu lửa", "toa tàu"],
        "patterns": [r"tàu\s+hỏa", r"đường\s+sắt", r"tàu\s+lửa", r"toa\s+tàu"],
        "priority": 9
    },
    "Tàu thủy, thuyền": {
        "keywords": ["tàu thủy", "thuyền", "boat", "ship", "thủy"],
        "patterns": [r"tàu\s+thủy", r"thuyền"],
        "priority": 8
    },
    
    # Electric and special transport
    "Xe điện": {
        "keywords": ["xe điện", "tram", "xe buýt điện", "xe máy điện"],
        "patterns": [r"xe\s+điện", r"xe\s+buýt\s+điện", r"xe\s+máy\s+điện"],
        "priority": 8
    },
    "Máy kéo": {
        "keywords": ["máy kéo", "tractor"],
        "patterns": [r"máy\s+kéo"],
        "priority": 8
    }
}

# Business and administrative patterns
business_patterns = {
    "Kinh doanh vận tải": {
        "keywords": ["kinh doanh vận tải", "vận tải", "kinh doanh", "doanh nghiệp vận tải"],
        "patterns": [r"kinh\s+doanh\s+vận\s+tải", r"doanh\s+nghiệp\s+vận\s+tải"],
        "priority": 9
    },
    "Đào tạo lái xe": {
        "keywords": ["đào tạo", "học lái xe", "trung tâm đào tạo", "cơ sở đào tạo"],
        "patterns": [r"đào\s+tạo", r"học\s+lái\s+xe", r"trung\s+tâm\s+đào\s+tạo", r"cơ\s+sở\s+đào\s+tạo"],
        "priority": 9
    },
    "Sát hạch lái xe": {
        "keywords": ["sát hạch", "thi bằng lái", "kiểm tra lái xe"],
        "patterns": [r"sát\s+hạch", r"thi\s+bằng\s+lái", r"kiểm\s+tra\s+lái\s+xe"],
        "priority": 9
    }
}

# Fallback categories
fallback_categories = {
    "Vi phạm giấy tờ": ["giấy phép", "bằng lái", "đăng ký", "chứng nhận"],
    "Vi phạm tốc độ": ["tốc độ", "chạy quá", "km/h", "vượt tốc độ"],
    "Vi phạm tín hiệu giao thông": ["đèn đỏ", "tín hiệu", "hiệu lệnh", "biển báo"],
    "Vi phạm về rượu bia": ["rượu", "bia", "cồn", "nồng độ cồn"],
    "Sử dụng điện thoại": ["điện thoại", "di động", "phone"],
    "Vi phạm dừng đỗ xe": ["dừng xe", "đỗ xe", "parking"],
    "Vi phạm mũ bảo hiểm": ["mũ bảo hiểm", "helmet"],
    "Vi phạm dây an toàn": ["dây an toàn", "thắt dây", "seat belt"],
    "Vi phạm chở người/hàng": ["chở người", "chở hàng", "quá tải", "overload"],
    "Vi phạm vượt xe": ["vượt xe", "vượt", "overtaking"],
    "Quản lý nhà nước": ["cơ quan", "thanh tra", "kiểm tra", "quản lý nhà nước"]
}

In [7]:
import re

def categorize_article(text, vehicle_patterns, business_patterns, fallback_categories):
    """
    Phân loại bài viết dựa trên text nội dung và các patterns định sẵn.
    Thứ tự ưu tiên: Vehicle -> Business -> Fallback
    """
    if not text:
        return ["Chưa xác định"]

    # Chuẩn hóa text để tìm kiếm chính xác hơn
    text_lower = text.lower()
    
    # --- 1. KIỂM TRA VEHICLE PATTERNS (Ưu tiên cao nhất) ---
    found_vehicles = []
    
    # Sắp xếp patterns theo priority (độ ưu tiên) từ cao xuống thấp trước khi loop
    # (Để đảm bảo các loại xe quan trọng được check trước nếu cần thiết)
    sorted_vehicles = sorted(vehicle_patterns.items(), key=lambda item: item[1]['priority'], reverse=True)
    
    for category, data in sorted_vehicles:
        # Kiểm tra Regex patterns
        for pattern in data['patterns']:
            if re.search(pattern, text_lower, re.IGNORECASE):
                found_vehicles.append(category)
                return found_vehicles

    # --- 2. KIỂM TRA BUSINESS PATTERNS (Ưu tiên nhì) ---
    found_business = []
    sorted_business = sorted(business_patterns.items(), key=lambda item: item[1]['priority'], reverse=True)
    
    for category, data in sorted_business:
        for pattern in data['patterns']:
            if re.search(pattern, text_lower, re.IGNORECASE):
                found_business.append(category)
                
                return found_business

    # --- 3. KIỂM TRA FALLBACK CATEGORIES (Ưu tiên cuối) ---
    found_fallback = []
    
    for category, keywords in fallback_categories.items():
        for keyword in keywords:
            # Kiểm tra keyword xuất hiện trong text (matching chuỗi đơn giản)
            if keyword.lower() in text_lower:
                found_fallback.append(category)
    
                return found_fallback

    # --- 4. MẶC ĐỊNH ---
    return ["Khác"]

# process ND100

In [8]:
raw_text = read_input_file("/home/taidvt/vietnamese-traffic-law-qa/docs/nd100_2019.pdf")

📂 Đang đọc file nguồn: /home/taidvt/vietnamese-traffic-law-qa/docs/nd100_2019.pdf...
   📄 File PDF có 102 trang.
✅ Đã đọc xong văn bản nguồn.


In [9]:
articles = split_into_articles(raw_text)
articles

['CHÍNH PHỦ\n-------\nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\n---------------\nSố: 100/2019/NĐ-CP Hà Nội, ngày 30 tháng 12 năm 2019\nNGHỊ ĐỊNH\nQUY ĐỊNH XỬ PHẠT VI PHẠM HÀNH CHÍNH TRONG LĨNH VỰC GIAO THÔNG ĐƯỜNG BỘ\nVÀ ĐƯỜNG SẮT\nCăn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015;\nCăn cứ Luật Xử lý vi phạm hành chính ngày 20 tháng 6 năm 2012;\nCăn cứ Luật Giao thông đường bộ ngày 13 tháng 11 năm 2008;\nCăn cứ Luật Đường sắt ngày 16 tháng 6 năm 2017;\nTheo đề nghị của Bộ trưởng Bộ Giao thông vận tải;\nChính phủ ban hành Nghị định quy định xử phạt vi phạm hành chính trong lĩnh vực giao thông đường\nbộ và đường sắt.\nChương I\nNHỮNG QUY ĐỊNH CHUNG\n',
 'Điều 1. Phạm vi điều chỉnh\n1. Nghị định này quy định về hành vi vi phạm hành chính; hình thức, mức xử phạt, biện pháp khắc phục\nhậu quả đối với từng hành vi vi phạm hành chính; thẩm quyền lập biên bản, thẩm quyền xử phạt, mức\nphạt tiền cụ thể theo từng chức danh đối với hành vi vi phạm hành chính trong lĩnh

In [10]:
# get only the first line of each article
for idx in range(len(articles)):
    articles[idx] = articles[idx].split('\n')[0]
    

In [11]:
import json
with open("/home/taidvt/vietnamese-traffic-law-qa/data/processed/ND100_2019_normalized.json", "r") as f:
    data_100 = json.load(f)

for data_idx in range(len(data_100['violations'])):
    base_violation = data_100['violations'][data_idx]
    article = data_100['violations'][data_idx]['legal_basis']['article']
    if 'điều' not in article.lower():
        article = 'Điều ' + article
    for content in articles:
        if f"{article}." in content:
            print(content)
            break
    break

article2category = {}
for content in articles:
    cats = categorize_article(content, vehicle_patterns, business_patterns, fallback_categories)
    article_num = content.split('.')[0]
    article2category[article_num] = cats[0]

Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông


In [12]:
with open("/home/taidvt/vietnamese-traffic-law-qa/data/processed/article2category_ND100.json", "w") as f:
    json.dump(article2category, f, indent=4, ensure_ascii=False)


# process ND168

In [ ]:
raw_text = read_input_file("./docs/nd168_2024.pdf")

📂 Đang đọc file nguồn: /home/taidvt/vietnamese-traffic-law-qa/docs/nd168_2024.pdf...
   📄 File PDF có 77 trang.
✅ Đã đọc xong văn bản nguồn.


In [19]:
articles = split_into_articles(raw_text)

In [20]:
articles

['CHÍNH PHỦ\n-------\nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\n---------------\nSố: 168/2024/NĐ-CP Hà Nội, ngày 26 tháng 12 năm 2024\n \nNGHỊ ĐỊNH\nQUY ĐỊNH XỬ PHẠT VI PHẠM HÀNH CHÍNH VỀ TRẬT TỰ, AN TOÀN GIAO THÔNG TRONG\nLĨNH VỰC GIAO THÔNG ĐƯỜNG BỘ; TRỪ ĐIỂM, PHỤC HỒI ĐIỂM GIẤY PHÉP LÁI XE\nCăn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015; Luật sửa đổi, bổ sung một số điều của Luật\nTổ chức Chính phủ và Luật Tổ chức chính quyền địa phương ngày 22 tháng 11 năm 2019;\nCăn cứ Luật Xử lý vi phạm hành chính ngày 20 tháng 6 năm 2012; Luật sửa đổi, bổ sung một số điều\ncủa Luật Xử lý vi phạm hành chính ngày 13 tháng 11 năm 2020;\nCăn cứ Luật Trật tự, an toàn giao thông đường bộ ngày 27 tháng 6 năm 2024;\nTheo đề nghị của Bộ trưởng Bộ Công an;\nChính phủ ban hành Nghị định quy định xử phạt vi phạm hành chính về trật tự, an toàn giao thông trong\nlĩnh vực giao thông đường bộ; trừ điểm, phục hồi điểm giấy phép lái xe.\nChương I\nNHỮNG QUY ĐỊNH CHUNG\n',
 'Điều 1. 

In [ ]:
# get only the first line of each article
for idx in range(len(articles)):
    articles[idx] = articles[idx].split('\n')[0]
    

# create a mapping from article to category

In [24]:
import json
with open("/home/taidvt/vietnamese-traffic-law-qa/data/processed/ND168_2024_normalized.json", "r") as f:
    data_168 = json.load(f)

for data_idx in range(len(data_168['violations'])):
    base_violation = data_168['violations'][data_idx]
    article = data_168['violations'][data_idx]['legal_basis']['article']
    if 'điều' not in article.lower():
        article = 'Điều ' + article
    for content in articles:
        if f"{article}." in content:
            print(content)
            break
    break

article2category = {}
for content in articles:
    cats = categorize_article(content, vehicle_patterns, business_patterns, fallback_categories)
    article_num = content.split('.')[0]
    article2category[article_num] = cats[0]

Điều 6. Xử phạt, trừ điểm giấy phép lái xe của người điều khiển xe ô tô, xe chở người bốn bánh có


In [26]:
with open("/home/taidvt/vietnamese-traffic-law-qa/data/processed/article2category.json", "w") as f:
    json.dump(article2category, f, indent=4, ensure_ascii=False)
